In [1]:
BASE_DIR="/home/ec2-user/SageMaker"
SAGEMAKER_DIR = "/home/ec2-user/SageMaker/neural_nets_memorization"

In [2]:
%%time
#%%capture 
#os.path.join(SAGEMAKER_DIR,"neural_nets_memorization")
!git pull


Already up to date.
CPU times: user 102 ms, sys: 10.5 ms, total: 112 ms
Wall time: 5.08 s


In [3]:
%%time
%%capture
! pip install --upgrade {SAGEMAKER_DIR}
#!pip install matplotlib==3.1.3

CPU times: user 2.96 s, sys: 2.58 s, total: 5.53 s
Wall time: 1min 32s


In [4]:
from neural_nets_memorization import params
from neural_nets_memorization import utils
from neural_nets_memorization import dataset_management
from neural_nets_memorization import unbalanced_datasets
from neural_nets_memorization import privacy
from neural_nets_memorization import model_training
from neural_nets_memorization import models
from neural_nets_memorization import pate
from neural_nets_memorization import dp_prototypicalness
from neural_nets_memorization import memorization_estimator

import os 
import gc
import torch
import torch.nn as nn
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, Subset
import numpy as np
import datetime

import numba
from numba import jit, njit, prange, bool_, int_

from multiprocessing import Pool

from matplotlib import pyplot as plt 
import inspect

In [5]:
#print(inspect.getsource(model_training.train_model))

In [6]:
(trainset, testset), (trainloader,
                      testloader) = dataset_management.load_MNIST_datasets_and_loaders()# load_CIFAR10_datasets_and_loaders()
num_classes =10
device = params.get_default_device()
epochs = 7

In [8]:
eps = 400.
# 40 was 94 
a = model_training.train_model(
    model= models.CNN_model_factory(),
    model_name="test",
    epochs=epochs,
    train_loader= trainloader,
    device = device,
    test_loader=testloader,  # can be None - only called in per_epoch_callback hook
    max_grad_norm=params.MAX_GRAD_NORM,
    tensorboard_path=".",
    criterion = params.criterion,
    eps=eps,
    delta=1e-5,#None,
    per_epoch_callbacks=model_training.log_everything_callbacks,
    save_path=None,
    overwrite=False, verbose= True)


epsilon - 400.0
Using sigma=0.15641113281250002 and C=0.3
new data loader created <class 'torch.utils.data.dataloader.DataLoader'>
Training epoch: 0/7
privacy used  41.943469841947866
privacy used  151.865117931495
Epoch [0], 
	Accuracy: approx_train_accuracy: 0.7977, train_accuracy: 0.9404, val_accuracy: 0.9478
	Loss: approx_train_loss: 0.9722, train_loss: 0.3505, val_loss: 0.2940
time for epoch 0 : 0:00:39.086671
Training epoch: 1/7
privacy used  41.943469841947866
privacy used  193.31954708080784
Epoch [1], 
	Accuracy: approx_train_accuracy: 0.8691, train_accuracy: 0.9520, val_accuracy: 0.9542
	Loss: approx_train_loss: 0.5693, train_loss: 0.2811, val_loss: 0.2505
time for epoch 1 : 0:00:38.812214
Training epoch: 2/7
privacy used  41.943469841947866
privacy used  232.8935691346879
Epoch [2], 
	Accuracy: approx_train_accuracy: 0.8744, train_accuracy: 0.9512, val_accuracy: 0.9555
	Loss: approx_train_loss: 0.5534, train_loss: 0.2738, val_loss: 0.2456
time for epoch 2 : 0:00:38.461610
Tr

In [ ]:
# todo : print eps
%load_ext tensorboard
%tensorboard --logdir /home/ec2-user/SageMaker/neural_nets_memorization/notebooks/

# go to https://neural-nets-gpu-instance.notebook.us-east-1.sagemaker.aws/proxy/6006/#scalars

In [ ]:
eps = 2.
model_ =models.CNN_model_factory()
results, model_predictions = memorization_estimator.run_single_experiment(
                          model = model_,
                          dataloader = trainloader,
                          testloader=testloader,
                          memorization_dataloader = trainloader,
                          save_path =None ,
                          tensorboard_path=".",
                          model_name="CNN",
                          epochs=epochs,
                          device=params.get_default_device(),
                          N=60000,
                          epsilon=eps,
                          overwrite=False)


In [ ]:
params.get_default_device()

In [ ]:
model_ =models.CNN_model_factory()
results, model_predictions = memorization_estimator.run_single_experiment(
                          model = model_,
                          dataloader = trainloader,
                          testloader=testloader,
                          memorization_dataloader = trainloader,
                          save_path =None ,
                          tensorboard_path=".",
                          model_name="CNN",
                          epochs=epochs,
                          device=params.get_default_device(),
                          N=60000,
                          epsilon=5.0,
                          overwrite=False)


In [ ]:
N = 60000
m_ratio = 0.8
trials = 2000

mem_exp_masks = memorization_estimator.set_up_memorization_experiment_dataset_masks(
        filepath=None, N=N, m_ratio=m_ratio, trials=trials, overwrite=False)


In [ ]:
memorization_estimator.run_all_experiments(mem_experiment_masks=mem_exp_masks,
                        base_save = ".",
                        predictions_filepath="predictions.txt",
                        memorization_experiments_meta_filepath="ignore.txt",
                        dataset = trainset,
                        memorization_dataloader = trainloader,
                        epochs=5,
                        model_factory=models.CNN_model_factory,
                        device=params.get_default_device(),
                        predictions=None,
                        memorization_experiments_meta=None,
                        epsilon=3.,
                        max_grad_norm=params.MAX_GRAD_NORM,
                        overwrite=False)

In [ ]:
trainloader.dataset.targets

In [ ]:
model_training.get_predictions(model= model_, data_loader= trainloader, device = params.get_default_device())

In [ ]:
model_no_dp =models.CNN_model_factory()

results, model_predictions = memorization_estimator.run_single_experiment(
                          model = model_no_dp,
                          dataloader = trainloader,
                          testloader=None,
                          memorization_dataloader = trainloader,
                          save_path =None ,
                          tensorboard_path=None,
                          model_name="CNN",
                          epochs=5,
                          device=params.get_default_device(),
                          N=60000,
                          epsilon=None,
                          overwrite=False)


In [ ]:
from opacus.utils.batch_memory_manager import BatchMemoryManager
model = models.CNN_model_factory()


privacy_engine, model, optimizer, data_loader = privacy.privatize(model=model, 
                                                                  data_loader=trainloader, 
                                                                  epochs=10, epsilon= 1., delta=1.e-5, max_grad_norm=1., device= device)
    



with BatchMemoryManager(data_loader=trainloader, max_physical_batch_size=2, optimizer=optimizer) as new_data_loader:
    for data, label in new_data_loader:
        print(data)
        break


In [ ]:
# Lookie here # https://jovian.ai/roubish/final-course-assignment
def train_model(
        *,
        model,
        model_name="",
        epochs,
        train_loader,
        device,
        test_loader=None,  # can be None - only called in per_epoch_callback hook
        max_grad_norm=params.MAX_GRAD_NORM,
        tensorboard_path=None,
        criterion,
        eps=None,
        delta=None,
        per_epoch_callbacks=None,
        save_path=None,
        overwrite=False):
    torch.cuda.empty_cache()
    # set up
    first_start = datetime.datetime.now()
    estimated_total_correct = 0

    history = []

    ret = {
        "save_path": save_path,
        "privacy": {
            "epsilon": eps,
            "delta": delta
        },
        "history": history,
        "model_name": model_name
    }
    print(f"epsilon - {eps}")
    if save_path is not None and os.path.exists(save_path) and not overwrite:
        print(f"path {save_path} already saved to. SKIPPING.")
        return ret
    writer = SummaryWriter(
        tensorboard_path) if tensorboard_path is not None else None

    per_epoch_callbacks = per_epoch_callbacks or []
    # set up privacy experiment if needed (use memory safe data loader as needed)
    dataloader = train_loader
    optimizer = None
    if eps is not None:
        
        _, model, optimizer, train_loader = privacy.privatize(
            model=model,
            data_loader=train_loader,
            epochs=epochs,
            epsilon=eps,
            delta=delta,
            max_grad_norm=max_grad_norm,
            device=device)

    model = model.to(device)
    print(optimizer)
    print("do we exist?")
    optimizer = optimizer or utils.get_new_optimizer(model) 

    # create writer

    def _train(dataloader):
        """ Helper to train either with DP or not (creates closure)"""
        for epoch in range(epochs):
            start = datetime.datetime.now()
            print(f"Training epoch: {epoch}/{epochs}")
            estimated_total_correct, estimated_epoch_train_losses = model_training.train_single_epoch(
                model=model,
                train_loader=dataloader,
                optimizer=optimizer,
                criterion=criterion,
                device=device)

            result = {
                "epoch":
                epoch,
                "approx_train_loss":
                np.mean(estimated_epoch_train_losses),
                "approx_train_accuracy":
                estimated_total_correct / len(dataloader.dataset)
            }  # todo : do something more with result

            # call all callbacks

            for callback in per_epoch_callbacks:  # to do : check if callback is called on the last batch
                callback(model=model,
                         epoch=epoch,
                         train_loader=dataloader,
                         test_loader=test_loader,
                         optimizer=optimizer,
                         criterion=criterion,
                         result=result,
                         writer=writer,
                         device=device)
            history.append(result)
            epoch_end(result=result)
            print(
                f"time for epoch {epoch} : {(datetime.datetime.now() - start)}"
            )

    # train with DP
    if eps is None:
        # train without DP
        _train(dataloader)
    else:
        with BatchMemoryManager(
                data_loader=train_loader,
                max_physical_batch_size=model_training.MAX_PHYSICAL_BATCH_SIZE,
                optimizer=optimizer) as new_data_loader:
            print(f"new data loader created {type(new_data_loader)}")
            _train(new_data_loader)

    if save_path is not None:
        utils.save_model(model, save_path)

    print(f"total time: {(datetime.datetime.now() - first_start)}")
    if writer is not None:
        writer.flush()
    return ret




train_model(
        model= models.CNN_model_factory(),
        model_name="test",
        epochs=4,
        train_loader= trainloader,
        device = device,
        test_loader=None,  # can be None - only called in per_epoch_callback hook
        max_grad_norm=params.MAX_GRAD_NORM,
        tensorboard_path=None,
        criterion = params.criterion,
        eps=4.,
        delta=1e-5,#None,
        per_epoch_callbacks=None,
        save_path=None,
        overwrite=False)

In [ ]:
def get_weighted_indices(dataset, class_ordering, class_distribution_counts,
                         class_to_dataset):
    """
    class_ordering : a list continaing numbers 0-9 (inclusive), in some order
    distribution_counts : a list denoting the number of points to sample from a particular class
    class_to_dataset : a dictionary mapping class-id to a dataset of only data from that class

    returns a dictionary of lists of indices according to class_ordering and class_distribution_counts
    """
    indices = {}

    for i, class_id in enumerate(class_ordering):
        og_class_dataset = class_to_dataset[class_id]
        og_class_indices = class_to_dataset[class_id].indices.T.numpy(
        ).nonzero()[0]
        #og_class_dataset.indices.nonzeros()

        number_of_points_from_class = int(
            min(class_distribution_counts[i], len(og_class_dataset)))
        # randomly pick a subset of the data points of a particular class
        indices[class_id] = np.random.choice(og_class_indices,
                                             size=number_of_points_from_class,
                                             replace=False)
    return indices

In [ ]:

number_of_datasets =num_teachers = 30

class_ordering = np.arange(len(trainset.classes))

        
class_to_dataset = unbalanced_datasets.get_class_to_dataset(trainset)
class_to_dataset

In [ ]:
c =0
for i in class_to_dataset[0].dataset:
    c+=1
#(dataset.targets[dataset.targets==class_id],dataset.data[dataset.targets==class_id] )
c

In [ ]:
ds = class_to_dataset[0]
ds.

In [ ]:
class_to_dataset = unbalanced_datasets.get_class_to_dataset(trainset)
class_distribution_counts = unbalanced_datasets.get_class_distribution_counts_from_dataset(trainset)

weighted_indices = get_weighted_indices(trainset, class_ordering, class_distribution_counts, class_to_dataset)

plt.plot(np.arange(len(class_distribution_counts)),
             class_distribution_counts)
plt.title("class sizes of the weighted datasets")
plt.show()
